## get_stock_correlations - 
1. Use the barchart api to retrieve stock historical data;
2. Create a correlation matrix from the historical data; 
3. Get standard deviations from the historical data;
4. Create long/short position information for each security retrieved
5. Create a portolio VaR using all of the above

## Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from os.path import expanduser
from scipy.stats import norm
from numpy import linalg as LA

### Add to sys.path
Within the barchart_api module, references like: ```from barchartapi import barchart_api``` will not work unless you append to sys.path the following folders:
1. project working directory (which holds barchart_api.py),
2. project directory,
3. workspace directory.

In [ ]:
WORKSPACE_DIR = '../..' 
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' 
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' 
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
import barchart_api as bcapi

___
### Create paths to where you will store the history data

In [ ]:
# DROPBOX_MARKET_DATA_DIR = expanduser('~/Dropbox/market_data')
# STOCKS_DIR = f'{DROPBOX_MARKET_DATA_DIR}/stocks'
# !ls {STOCKS_DIR}
STOCKS_DIR = f'./temp_folder/stocks'
try:
    os.makedirs(STOCKS_DIR)
except:
    pass

___
### Create an instance of BcHist
1. must set bar_type to minutes, hour or daily
2. must set interval
 * for minutes, set interval to 1, 5, 15, 30 or 60
 * for other types, see https://www.barchart.com/ondemand/api/getHistory

In [ ]:
# set this to 'free' or 'paid'
endpoint = 'free' # free or paid

# set the bar_type and the interval
bar_type='minutes' # minutes, daily, monthly
interval=30 # 1,5,15,30,60

# create an instance 
api_key = open(f'./temp_folder/{endpoint}_api_key.txt','r').read()
endpoint_type=f'{endpoint}_url'
bch = bcapi.BcHist(api_key, bar_type=bar_type, interval=interval,endpoint_type = endpoint_type)


In [ ]:
def get_portfolio(csv_path=None):
    '''
    fetch a portfolio to use below
    '''
    p = csv_path
    if p is None:
        p = './hi_volume_stocks.csv'
    df_p = pd.read_csv(p)
    df_p['under'] = df_p.symbol.apply(lambda s: s.split("_")[0])    
    pnames = list(df_p.under)
    return (df_p,pnames)

### Get your portfolio
(default is ./hi_volume_stocks.csv)

In [ ]:
p_tuple = get_portfolio()
df_portfolio = p_tuple[0]

___
### Main Loop to fetch history data from Barchart

#### Inputs
On each call to BcHist.get_history, provide:
 * a beginning yyyymmdd integer
 * an ending yyyymmdd integer
 * a short name like:
  * a stock symbol like: SPY,USO,IBM, etc
  * a futures symbol like: CLJ18, GCG19, ESH17, etc

#### Outputs
The get_history method will return a tuple, where:
 * tup[0] provides a status

* tup[1] provides a pandas DataFrame of data, or None

In [ ]:
# true will cause barchartapi to refetch csv data, 
#   false will cause it to bypass any short_name that already has a csv file in the path sn_path below
delete_old_csv_file = False 

#  set a date range
days_to_fetch = 120
dt_end = datetime.datetime.now()
dt_beg = dt_end - datetime.timedelta(days_to_fetch)
beg_yyyymmdd = '%04d%02d%02d' %(dt_beg.year,dt_beg.month,dt_beg.day)#20181201
end_yyyymmdd = '%04d%02d%02d' %(dt_end.year,dt_end.month,dt_end.day)#20190219


# set up other ETF names that can be used as a hedge to the portfolio
spydr_short_names = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF']
equity_etf_names = ['DIA','SPY','QQQ']
commodity_etf_short_names = ['USO','UNG','DBC','DBA','GLD','USCI']
currency_etf_short_names = ['FXY','FXE','FXB','FXF','FXC','FXA']
# my_portfolio_short_names = ['XLE','SPY','USO','GLD','XLU','QQQ']

# get the portfolio

# set short_names to the desired list to get
short_names = list(set(
    p_tuple[1] + spydr_short_names + equity_etf_names + commodity_etf_short_names + currency_etf_short_names + commodity_etf_short_names))

for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    if os.path.isfile(sn_path):
        if delete_old_csv_file:
            os.remove(sn_path)
        else:
            print(f'BYPASSING: {short_name}')
            continue
    print(f'get_history: {short_name} BEGIN {datetime.datetime.now()}')
    tup = bch.get_history(short_name, beg_yyyymmdd, end_yyyymmdd)
    print(f'get_history: {short_name} WRITING DATA {datetime.datetime.now()}')
    tup[1].to_csv(sn_path,index=False)
    print(f'get_history: {short_name} END {datetime.datetime.now()}')
    

___
### Do something interesting with the returned data, like create a portfolio VaR

In [ ]:
df_hist = None
for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    df_temp = pd.read_csv(sn_path)
    df_temp = df_temp[['timestamp','close']]
    df_temp = df_temp.rename(columns={'close':f'{short_name}'})
    if df_hist is None:
        df_hist = df_temp.copy()
    else:
        df_hist = df_hist.merge(df_temp,how='inner',on='timestamp')
df_corr = df_hist[df_portfolio.under].corr()
df_corr

In [ ]:
cols = list(set(list(df_hist.columns.values))-set(['timestamp']))
bars_per_day = 8*2
perc_of_day = 1/bars_per_day
perc_of_year = perc_of_day/256
std_series = df_hist[cols].pct_change().iloc[1:].std()/perc_of_year**.5
df_std = pd.DataFrame({'stdev':list(std_series.values),'symbol':list(std_series.index.values)})
df_std.sort_values('symbol')

#### Enter Positions

In [ ]:
df_positions = df_portfolio[['under','position']].rename(columns={'under':'symbol'})
df_positions

#### Get current prices

In [ ]:
cols = list(filter(lambda c:'time' not in c,df_hist.columns.values))
vals = df_hist.iloc[-1:][cols].as_matrix().reshape(-1)
df_prices = pd.DataFrame({'symbol':cols,'price':vals})[['symbol','price']]
df_prices

#### Merge position, prices, stds

In [ ]:
df_positions_2 = df_positions.merge(df_prices,how='inner',on='symbol')
df_positions_3 = df_positions_2.merge(df_std,how='inner',on='symbol')
df_positions_3

#### Create position VaR's

In [ ]:
VAR_CONFIDENCE = .99

In [ ]:
df_positions_3['unit_var'] = df_positions_3.apply(lambda r: r.price * r.stdev * norm.ppf(VAR_CONFIDENCE) * (1/256)**.5 / r.price,axis=1 )
df_positions_3['position_var'] = df_positions_3.apply(lambda r: r.unit_var * r.position * r.price ,axis=1 )
df_positions_3

#### Create portfolio VaR

In [ ]:
# create an spy standard deviation that is the historical average
var_days = 1
spy_usual_daily_std = .16 
port_std = (df_positions_3.position_var.astype(float).as_matrix().T @ df_corr.astype(float).as_matrix() @ df_positions_3.position_var.astype(float).as_matrix())**.5
port_var = port_variance**.5 * var_days**.5  * norm.ppf(VAR_CONFIDENCE) 
spy_curr_unit_var = float(df_std[df_std.symbol=='SPY'].stdev) * (var_days/256)**.5  * norm.ppf(VAR_CONFIDENCE) 
sp_dollar_equiv = port_var / spy_curr_unit_var 



In [ ]:
print(f'portolio VaR: {round(port_var,2)}')
print(f"sp {var_days} day{'s' if var_days>1 else ''} unit VaR: {round(spy_curr_unit_var,2)}")
print(f'Equivalent S&P position (in dollars): {round(sp_dollar_equiv,2)}')


## END